# Creating a Bank Database
The object is to design a bank database in SQL Server and then run various complex queries.  All of the information is made up. 

In [1]:
# connecting to a local SQL Server database using the pyodbc Python library. Used * for user security.   
import pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=*;'
                      'Trusted_connection=*;'
                      'username=*;'
                       'dbname=*')

In [2]:
# creating a cursor object.
cur = conn.cursor()

In [3]:
# creating a function in Python using the Pandas library to read the results into a dataframe.  Easier to read and analyze.
import pandas as pd
def read_query(q):
    read = pd.read_sql(q, conn)
    return read

In [13]:
# creating the first table named branch.
cur.execute('''CREATE TABLE branch (
                branch_number SMALLINT IDENTITY PRIMARY KEY,
                branch_name VARCHAR(30) NOT NULL UNIQUE,
                state CHAR(2) NOT NULL,
                city VARCHAR(30) NOT NULL
                );
                ''')

In [14]:
# inserting values inside of the table branch.
cur.execute('''INSERT INTO branch (branch_name, state, city) VALUES 
                ('Uptown', 'IL','Chicago'),
                ('Hickory', 'IL','Hickory Hills'),
                ('Newport', 'IN','Jameson'),
                ('Johnston','IN', 'Merryville'),
                ('New_berry','IA','Des Moines');
                ''')

In [7]:
read_query("SELECT * FROM branch;")

,branch_number,branch_name,state,city
0,1,Uptown,IL,Chicago
1,2,Hickory,IL,Hickory Hills
2,3,Newport,IN,Jameson
3,4,Johnston,IN,Merryville
4,5,New_Berry,IA,Des Moines


In [16]:
# creating a second table with employee information.
cur.execute('''CREATE TABLE employee (
               employee_id SMALLINT IDENTITY PRIMARY KEY,
               first_name VARCHAR(40) NOT NULL,
               last_name VARCHAR(40) NOT NULL,
               salary INT NOT NULL,
               years_at_company TINYINT,
               branch_number SMALLINT NOT NULL,
               supervisor_id TINYINT,
               title VARCHAR(30) NOT NULL,
               FOREIGN KEY (branch_number) REFERENCES branch(branch_number)
               );
            ''')

In [18]:
# inserting data into the employee table.
cur.execute('''INSERT INTO employee (first_name, last_name, salary, years_at_company, branch_number, supervisor_id, 
                title) VALUES
                ('John','Sanders', 100000, 15, 1, NULL, 'Regional Manager'),
                ('Sarah','Summers', 60000, 8, 1, 1, 'Branch Manager'),
                ('Jacob', 'Machlione', 45000, 4, 1, 2, 'Banker'),
                ('Justin' ,'Smith', 70000, 9, 2, 1, 'Branch Manager'),
                ('Joline', 'Murphy', 35000, 2, 2, 4, 'Banker'),
                ('Michael','Lichione', 50000, 10, 3, 1,'Branch Manager'),
                ('Jessica','Samuels', 42000, 2, 3, 6, 'Banker'),
                ('Nick', 'Linden', 63000, 6, 4, 1, 'Branch Manager'),
                ('Karen', 'Nunes', 50000, 7, 4, 8, 'Banker'),
                ('Dominic', 'Diago', 80000, 12, 5, 1, 'Branch Manager'),
                ('Monica', 'Santiago', 45000, 15, 5, 10, 'Banker');
                ''')

In [19]:
conn.commit()

In [5]:
read_query("SELECT TOP 5 * FROM employee;") # selecting the first 5 rows. 

,employee_id,first_name,last_name,salary,years_at_company,branch_number,supervisor_id,title
0,1,John,Sanders,100000,15,1,NaN,Regional Manager
1,2,Sarah,Summers,60000,8,1,1.0,Branch Manager
2,3,Jacob,Machlione,45000,4,1,2.0,Banker
3,4,Justin,Smith,70000,9,2,1.0,Branch Manager
4,5,Joline,Murphy,35000,2,2,4.0,Banker


# Recursive/Other Complex Queries

In [44]:
# recursive query displaying the employee path hierarchy.
read_query('''WITH hierarchy AS (
              SELECT
              employee_id,
              first_name,
              last_name,
              supervisor_id,
              CAST('Boss' AS TEXT) AS hierarchy_path
              FROM employee
              WHERE supervisor_id IS NULL 
              
              UNION ALL
              
              SELECT
              e2.employee_id,
              e2.first_name,
              e2.last_name,
              e2.supervisor_id,
              CAST(CONCAT(h.hierarchy_path, ' ', '>', ' ', e2.last_name) AS TEXT) AS hierarchy_path
              FROM employee e2
              INNER JOIN hierarchy h
              ON e2.supervisor_id = h.employee_id
              )
              
              SELECT * FROM hierarchy;
              ''')

,employee_id,first_name,last_name,supervisor_id,hierarchy_path
0,1,John,Sanders,NaN,Boss
1,2,Sarah,Summers,1.0,Boss > Summers
2,4,Justin,Smith,1.0,Boss > Smith
3,6,Michael,Lichione,1.0,Boss > Lichione
4,8,Nick,Linden,1.0,Boss > Linden
5,10,Dominic,Diago,1.0,Boss > Diago
6,11,Monica,Santiago,10.0,Boss > Diago > Santiago
7,9,Karen,Nunes,8.0,Boss > Linden > Nunes
8,7,Jessica,Samuels,6.0,Boss > Lichione > Samuels
9,5,Joline,Murphy,4.0,Boss > Smith > Murphy


In [98]:
# creating 3 groups based on years of experience, finding the average salary of each experience group
# and then comparing it to the employee salary.  
read_query('''WITH grouping_years_of_experience AS (
              SELECT 
              CONCAT(first_name, ' ', last_name) AS employee_name,
              salary,
              years_at_company,
              CASE 
              WHEN years_at_company < 5 THEN 'entry level'
              WHEN years_at_company >= 5 AND years_at_company < 10 THEN 'middle level'
              WHEN years_at_company >= 10 THEN 'senior level'
              END as experience_group
              FROM employee
              ),
          
              avg_grouping_salary AS (
              SELECT
              employee_name,
              years_at_company,
              salary,
              experience_group,
              AVG(salary) OVER (PARTITION BY experience_group) AS experience_group_avg
              FROM grouping_years_of_experience
              )
          
              SELECT
              employee_name,
              years_at_company,
              experience_group,
              salary,
              experience_group_avg,
              CASE 
              WHEN salary > experience_group_avg THEN 'above average'
              WHEN salary <= experience_group_avg THEN 'below average'
              END AS salary_compared_to_experience_group_avg
              FROM avg_grouping_salary
              ORDER BY experience_group, years_at_company;
              ''')

,employee_name,years_at_company,experience_group,salary,experience_group_avg,salary_compared_to_experience_group_avg
0,Jessica Samuels,2,entry level,42000,40666,above average
1,Joline Murphy,2,entry level,35000,40666,below average
2,Jacob Machlione,4,entry level,45000,40666,above average
3,Nick Linden,6,middle level,63000,60750,above average
4,Karen Nunes,7,middle level,50000,60750,below average
5,Sarah Summers,8,middle level,60000,60750,below average
6,Justin Smith,9,middle level,70000,60750,above average
7,Michael Lichione,10,senior level,50000,68750,below average
8,Dominic Diago,12,senior level,80000,68750,above average
9,Monica Santiago,15,senior level,45000,68750,below average


In [8]:
# window functions.  Average employee salary by branch and running sum of the salary by branch. 
read_query('''SELECT
              employee_id,
              CONCAT(first_name, ' ', last_name) AS employee_name,
              salary,
              branch_number,
              AVG(salary) OVER(PARTITION BY branch_number) AS avg_salary_by_branch,
              SUM(salary) OVER(PARTITION BY branch_number ORDER BY salary) AS running_branch_salary
              FROM employee;
              ''')

,employee_id,employee_name,salary,branch_number,avg_salary_by_branch,running_branch_salary
0,3,Jacob Machlione,45000,1,68333,45000
1,2,Sarah Summers,60000,1,68333,105000
2,1,John Sanders,100000,1,68333,205000
3,5,Joline Murphy,35000,2,52500,35000
4,4,Justin Smith,70000,2,52500,105000
5,7,Jessica Samuels,42000,3,46000,42000
6,6,Michael Lichione,50000,3,46000,92000
7,9,Karen Nunes,50000,4,56500,50000
8,8,Nick Linden,63000,4,56500,113000
9,11,Monica Santiago,45000,5,62500,45000


In [49]:
# showing the employee with their supervisor.  Combined the first and last names and ordered by supervisor_id.
read_query('''SELECT
              e.employee_id,
              CONCAT(e.first_name, ' ',  e.last_name) employee_name,
              e.title,
              e.supervisor_id,
              CONCAT(s.first_name, ' ', s.last_name) supervisor_name
              FROM employee e
              LEFT JOIN employee s
              ON e.supervisor_id = s.employee_id
              ORDER BY e.supervisor_id;
              ''')

,employee_id,employee_name,title,supervisor_id,supervisor_name
0,1,John Sanders,Regional Manager,NaN,
1,2,Sarah Summers,Branch Manager,1.0,John Sanders
2,4,Justin Smith,Branch Manager,1.0,John Sanders
3,6,Michael Lichione,Branch Manager,1.0,John Sanders
4,8,Nick Linden,Branch Manager,1.0,John Sanders
5,10,Dominic Diago,Branch Manager,1.0,John Sanders
6,3,Jacob Machlione,Banker,2.0,Sarah Summers
7,5,Joline Murphy,Banker,4.0,Justin Smith
8,7,Jessica Samuels,Banker,6.0,Michael Lichione
9,9,Karen Nunes,Banker,8.0,Nick Linden


In [20]:
# ranking employees by salary and finding the employees with the 5th and 8th highest salaries.
read_query('''WITH ranking AS (
              SELECT 
              first_name,
              last_name,
              salary,
              ROW_NUMBER() OVER(ORDER BY salary DESC) AS rank
              FROM employee
              )
              SELECT * FROM ranking
              WHERE rank IN(5, 8);
              ''')

,first_name,last_name,salary,rank
0,Sarah,Summers,60000,5
1,Jacob,Machlione,45000,8


In [99]:
conn.close() # closing the connection.